In [2]:
import os
import wfdb
from wfdb import processing

In [7]:
class HeartbeatSlicer:
    def __init__(self, record_path):
        self.record = wfdb.rdrecord(record_path)
        self.ecg_data = self.record.p_signal
        self.sampling_rate = self.record.fs
        self.peak_indices = self.detect_peaks()

    def detect_peaks(self):
        peak_indices = processing.gqrs_detect(sig=self.ecg_data, fs=self.sampling_rate)
        return peak_indices

    def get_peaks(self):
        return self.peak_indices

In [12]:
sliced = HeartbeatSlicer("data_original/mitdb/1.0.0/100")

In [13]:
print(sliced.get_peaks())

[    139     726    1311 ... 1298955 1299453 1299980]
